In [32]:
!pip install --upgrade pip
!pip install geonamescache unidecode
import requests,geonamescache,re,json
import pandas as pd
import numpy as np
import unidecode

Requirement already up-to-date: pip in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (20.0.2)


#Get data news headline text file from website and change to be string text#

In [33]:
Respond=requests.get('https://livevideo-resources.s3-us-west-2.amazonaws.com/course-data/93/headlines.txt')
Respond_text=Respond.text

#Spilt headline into list#

In [34]:
Headline_text=Respond_text.split('\n')
Headline_text[:4]

['Zika Outbreak Hits Miami',
 'Could Zika Reach New York City?',
 'First Case of Zika in Miami Beach',
 'Mystery Virus Spreads in Recife, Brazil']

Get list of countries

In [35]:
gc = geonamescache.GeonamesCache()
countries = [country["name"] for country in gc.get_countries().values()]
countries[:4]

['Andorra', 'United Arab Emirates', 'Afghanistan', 'Antigua and Barbuda']

Get list of cities

In [36]:
cities = [city['name'] for city in gc.get_cities().values()]
cities[:4]

['Andorra la Vella', 'Umm Al Quwain City', 'Ras Al Khaimah City', 'Zayed City']

Removing accent mark from the lists of countries and cities by unidecode

In [37]:
country_accent_mapping = {
    unidecode.unidecode(country): country for country in countries
}

city_accent_mapping = {
    unidecode.unidecode(city): city for city in cities
}

city_accent_mapping["Asmar"]

'Āsmār'

Removing in headline as well

In [38]:
Headline_text= [unidecode.unidecode(headline) for headline in Headline_text]
Headline_text[-4:]

['Suva authorities confirmed the spread of Rotavirus',
 'More Zika patients reported in Bella Vista',
 'Zika Outbreak in Wichita Falls',
 '']

Create list of unaccented countries and cities

In [39]:
unaccented_cities = list(city_accent_mapping.keys())
unaccented_countries = set(country_accent_mapping.keys())

print(f"There are {len(unaccented_cities)} cities to look through.")
print(f"There are {len(unaccented_countries)} countries to look through.")

There are 23022 cities to look through.
There are 252 countries to look through.


Sort Cities and Countries by Length

In [40]:
unaccented_cities = sorted(unaccented_cities, key=lambda x: len(x), reverse=True)
unaccented_cities[:2]

['Chak Two Hundred Forty-nine Thal Development Authority',
 'Dolores Hidalgo Cuna de la Independencia Nacional']

In [41]:
unaccented_countries = sorted(unaccented_countries, key=lambda x: len(x), reverse=True)
unaccented_countries[:2]

['South Georgia and the South Sandwich Islands',
 'United States Minor Outlying Islands']

Constructing the Regular Expressions

In [42]:
city_regex = r'\b|\b'.join(unaccented_cities)
city_regex[1500:1800]

'-Baume\\b|\\bTamuning-Tumon-Harmon Village\\b|\\bTultitlan de Mariano Escobedo\\b|\\bSan Bernardino Tlaxcalancingo\\b|\\bSan Francisco Tlalcilalcalpan\\b|\\bFraccionamiento Ciudad Olmeca\\b|\\bPresidencia Roque Saenz Pena\\b|\\bZurich (Kreis 11) / Oerlikon\\b|\\bSan Fernando de Monte Cristi\\b|\\bPuerto Francisco de '

In [43]:
country_regex = r"\b|\b".join(unaccented_countries)
country_regex[:100]

'South Georgia and the South Sandwich Islands\\b|\\bUnited States Minor Outlying Islands\\b|\\bBonaire, S'

Create city and country regular expression function

In [44]:
     """
    Find the city(s) and/or country(s) in a text headline.
    
    :param headline: string for headline
    
    :return dict: a dictionary mapping the headline to city(s) and/or countries.
     """
def find_city_and_country_in_headline(headline):
    city_match = re.search(city_regex, headline)
    country_match = re.search(country_regex, headline)
    cities = None if not city_match else city_match.group(0)
    countries = None if not country_match else country_match.group(0)
    return dict(headline=headline, countries=countries, cities=cities)

In [45]:
find_city_and_country_in_headline(Headline_text[3])

{'headline': 'Mystery Virus Spreads in Recife, Brazil',
 'countries': 'Brazil',
 'cities': 'Recife'}

In [46]:
find_city_and_country_in_headline(Headline_text[1])

{'headline': 'Could Zika Reach New York City?',
 'countries': None,
 'cities': 'New York City'}

Apply Regular Expression to All Headlines

In [47]:
headline_cities_and_countries = [
    find_city_and_country_in_headline(headline) for headline in Headline_text
]
headline_cities_and_countries[-10:]

[{'headline': 'More Zika patients reported in Fort Worth',
  'countries': None,
  'cities': 'Fort Worth'},
 {'headline': 'Zika symptoms spotted in Boynton Beach',
  'countries': None,
  'cities': 'Boynton Beach'},
 {'headline': 'Outbreak of Zika in Portoviejo',
  'countries': None,
  'cities': 'Portoviejo'},
 {'headline': 'Influenza Exposure in Muscat',
  'countries': None,
  'cities': 'Muscat'},
 {'headline': 'Rumors about Rabies spreading in Jerusalem have been refuted',
  'countries': None,
  'cities': 'Jerusalem'},
 {'headline': 'More Zika patients reported in Indang',
  'countries': None,
  'cities': 'Indang'},
 {'headline': 'Suva authorities confirmed the spread of Rotavirus',
  'countries': None,
  'cities': 'Suva'},
 {'headline': 'More Zika patients reported in Bella Vista',
  'countries': None,
  'cities': 'Bella Vista'},
 {'headline': 'Zika Outbreak in Wichita Falls',
  'countries': None,
  'cities': 'Wichita Falls'},
 {'headline': '', 'countries': None, 'cities': None}]

In [48]:
with open("headline_cities_and_countries.json", 'w') as outfile:
    json.dump(headline_cities_and_countries, outfile)

In [49]:
with open('headline_cities_and_countries.json', "r") as fin:
    check_data = json.loads(fin.read())

In [50]:
check_data[-10:]

[{'headline': 'More Zika patients reported in Fort Worth',
  'countries': None,
  'cities': 'Fort Worth'},
 {'headline': 'Zika symptoms spotted in Boynton Beach',
  'countries': None,
  'cities': 'Boynton Beach'},
 {'headline': 'Outbreak of Zika in Portoviejo',
  'countries': None,
  'cities': 'Portoviejo'},
 {'headline': 'Influenza Exposure in Muscat',
  'countries': None,
  'cities': 'Muscat'},
 {'headline': 'Rumors about Rabies spreading in Jerusalem have been refuted',
  'countries': None,
  'cities': 'Jerusalem'},
 {'headline': 'More Zika patients reported in Indang',
  'countries': None,
  'cities': 'Indang'},
 {'headline': 'Suva authorities confirmed the spread of Rotavirus',
  'countries': None,
  'cities': 'Suva'},
 {'headline': 'More Zika patients reported in Bella Vista',
  'countries': None,
  'cities': 'Bella Vista'},
 {'headline': 'Zika Outbreak in Wichita Falls',
  'countries': None,
  'cities': 'Wichita Falls'},
 {'headline': '', 'countries': None, 'cities': None}]

In [51]:
with open("country_accent_mapping.json", "w") as fout:
    json.dump(country_accent_mapping, fout)

In [52]:
with open("city_accent_mapping.json", "w") as fit:
    json.dump(city_accent_mapping, fit)

In [62]:
data = pd.read_json("headline_cities_and_countries.json")
data = data.replace({None: np.nan})
data=data[["headline","countries","cities"]]
data.head(10)

,headline,countries,cities
0,Zika Outbreak Hits Miami,NaN,Miami
1,Could Zika Reach New York City?,NaN,New York City
2,First Case of Zika in Miami Beach,NaN,Miami Beach
3,"Mystery Virus Spreads in Recife, Brazil",Brazil,Recife
4,Dallas man comes down with case of Zika,NaN,Dallas
5,Trinidad confirms first Zika case,NaN,Trinidad
6,Zika Concerns are Spreading in Houston,NaN,Houston
7,Geneve Scientists Battle to Find Cure,NaN,Geneve
8,The CDC in Atlanta is Growing Worried,NaN,Atlanta
9,Zika Infested Monkeys in Sao Paulo,NaN,Sao Paulo


In [63]:
data["latitude"]=np.nan
data["longitude"]=np.nan
data["countrycode"]=np.nan

In [66]:
data=data.drop(columns=["countries"])
data.head(10)

,headline,cities,latitude,longitude,countrycode
0,Zika Outbreak Hits Miami,Miami,NaN,NaN,NaN
1,Could Zika Reach New York City?,New York City,NaN,NaN,NaN
2,First Case of Zika in Miami Beach,Miami Beach,NaN,NaN,NaN
3,"Mystery Virus Spreads in Recife, Brazil",Recife,NaN,NaN,NaN
4,Dallas man comes down with case of Zika,Dallas,NaN,NaN,NaN
5,Trinidad confirms first Zika case,Trinidad,NaN,NaN,NaN
6,Zika Concerns are Spreading in Houston,Houston,NaN,NaN,NaN
7,Geneve Scientists Battle to Find Cure,Geneve,NaN,NaN,NaN
8,The CDC in Atlanta is Growing Worried,Atlanta,NaN,NaN,NaN
9,Zika Infested Monkeys in Sao Paulo,Sao Paulo,NaN,NaN,NaN


In [95]:
for index in data.index:
        test=0
        best_sf = max(gc.get_cities_by_name(data['cities'][test]),
                 key=lambda x: list(x.values())[0]['population'])
        sf_data = list(best_sf.values())[0]
        data['latitude'][index] = sf_data['latitude']
        data['longitude'][index] = sf_data['longitude']
        data['countrycode'][index]= sf_data['countrycode']
        test+=1

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [96]:
data.head(10)

,headline,cities,latitude,longitude,countrycode
0,Zika Outbreak Hits Miami,Miami,25.77427,-80.19366,US
1,Could Zika Reach New York City?,New York City,25.77427,-80.19366,US
2,First Case of Zika in Miami Beach,Miami Beach,25.77427,-80.19366,US
3,"Mystery Virus Spreads in Recife, Brazil",Recife,25.77427,-80.19366,US
4,Dallas man comes down with case of Zika,Dallas,25.77427,-80.19366,US
5,Trinidad confirms first Zika case,Trinidad,25.77427,-80.19366,US
6,Zika Concerns are Spreading in Houston,Houston,25.77427,-80.19366,US
7,Geneve Scientists Battle to Find Cure,Geneve,25.77427,-80.19366,US
8,The CDC in Atlanta is Growing Worried,Atlanta,25.77427,-80.19366,US
9,Zika Infested Monkeys in Sao Paulo,Sao Paulo,25.77427,-80.19366,US
